# Data prepare 

This segment prepares dataset containig normalized sound recordings extended with temperature and humidity values. For now we support only one hive for data preparation.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [16]:
# 1300001 - hive
# 1300002 - not hive?
# 1400001 - hive szymanski
# 1400002 - hive szymanski
#hives_ids = [1300001, 1300002, 1400001, 1400002]

hives_ids = [1300001, 1400001, 1400002]
DATA_INIT = False

night_start_hour = 21
night_end_hour = 5

In [5]:
if DATA_INIT:
    hives_temperature = [pd.read_csv(f"measurements/{hive_id}/humidity.csv") for hive_id in hives_ids]
    hives_humidity = [pd.read_csv(f"measurements/{hive_id}/temperature.csv") for hive_id in hives_ids]
    hives_humidity = [hive_hum.drop_duplicates(subset=['timestamp'], keep = False) for hive_hum in hives_humidity]
    hives_temperature = [hive_tem.drop_duplicates(subset=['timestamp'], keep = False) for hive_tem in hives_temperature]
    
    hives_atmosphere = [pd.merge(
        hives_humidity[idx], hives_temperature[idx], on='timestamp',
        suffixes=(f"_humidity_{hive_id}",f"_temperature_{hive_id}"))
               for idx, hive_id in enumerate(hives_ids)]
   
    total = 0
    for idx, hive_atmosphere in enumerate(hives_atmosphere):
        hive_atmosphere['timestamp'] = pd.to_datetime(
            hive_atmosphere['timestamp'], format = '%Y-%m-%dT%H-%M-%S').sort_values()
        hive_atmosphere.set_index('timestamp', inplace = True)
        print(f"Hive with id: {hives_ids[idx]} has atmospheric data with size: {hive_atmosphere.shape}")
        total += hive_atmosphere.shape[0]
    
    print(f"Total atmoshpere dataset size: {total}")

Hive with id: 1300001 has atmospheric data with size: (6696, 2)
Hive with id: 1400001 has atmospheric data with size: (2418, 2)
Hive with id: 1400002 has atmospheric data with size: (1936, 2)
Total atmoshpere dataset size: 11050


In [18]:
import glob
from scipy.fftpack import rfft
from scipy.signal import blackman
import librosa

if DATA_INIT:
    total_night_samples = 0
    hives_data = []
    for idx, hive_id in enumerate(hives_ids):
        sound_files = [f for f in glob.glob(f"measurements\\{hive_id}\\sound*.csv")]
        print(f"Sound data preparation for hive: {hive_id} which has {len(sound_files)} recordings")
        for file in tqdm(sound_files):
            df_samples = pd.read_csv(file)
            pd_timestamp = pd.to_datetime(file.split("sound-")[1].split(".csv")[0], format='%Y-%m-%dT%H-%M-%S')
            if(len(df_samples.index) == 3000 and max(df_samples.values) < 4500):
                np_samples = np.array(df_samples['samples'].values, dtype="float32")
                np_samples = np_samples / 4080
                
                # FFT, get rid of DC compoment by subtracting average
                w = blackman(3000)
                np_spectogram = rfft(w*(np_samples - np.mean(np_samples)))
                np_spectogram = np.abs(np_spectogram)
                
                # MFCC
                mfccs = librosa.feature.mfcc(y=np_samples, sr=3000, n_fft=512, hop_length=256, n_mfcc=14)
                np_mfcc_avg = np.mean(mfccs, axis=1)
                np_mfcc_var = np.var(mfccs, axis=1)

                # Night label
                is_night = (pd_timestamp.hour > night_start_hour or pd_timestamp.hour < night_end_hour)
                if is_night: 
                    total_night_samples = total_night_samples + 1
                
                # Atmosphere
                df_nerest = hives_atmosphere[idx].iloc[
                    hives_atmosphere[idx].index.get_loc(pd_timestamp, method='nearest')]
                
                hives_data.append([pd_timestamp, hive_id, is_night,
                                    df_nerest[f"value_temperature_{hive_id}"],
                                    df_nerest[f"value_humidity_{hive_id}"],
                                    np_samples, np_spectogram, np_mfcc_avg, np_mfcc_var])
    
    print(f'Creating dataset with {total_night_samples} night samples for data size: {len(hives_data)}')
    print(f'Night percentage: {round(100*total_night_samples/len(hives_data), 2)}')
    np.save('hives-data.npy', hives_data)
else:
    hives_data = np.load('hives-data.npy', allow_pickle = True)
    print(f"Loaded dataset with size {len(hives_data)}.")
    
df_hives_data = pd.DataFrame(hives_data,
                           columns=['timestamp', 'hive_id', 'is_night',
                                    'temperature', 'humidity',
                                    'samples', 'fft', 'mfcc_avg', 'mfcc_var'])
df_hives_data = df_hives_data.set_index('timestamp')

Loaded dataset with size 2936.


In [20]:
df_hives_data.head(100)

,hive_id,is_night,temperature,humidity,samples,fft,mfcc_avg,mfcc_var
timestamp,,,,,,,,
2019-08-02 20:06:39,1300001,False,56.93,34.21,"[0.5887255, 0.5865196, 0.5926471, 0.589951, 0....","[0.05721917318219877, 0.04508206282285264, 0.0...","[-358.4177321177189, 76.5997417203608, 7.55132...","[112.49110283839512, 97.55593689073913, 75.592..."
2019-08-02 20:21:37,1300001,False,56.93,34.09,"[0.5835784, 0.57916665, 0.57990193, 0.5901961,...","[0.11594218183563729, 0.10258726867713885, 0.1...","[-347.96401130058393, 83.9858585486864, 5.4264...","[123.00966802113419, 83.35277047728276, 37.325..."
2019-08-02 20:37:37,1300001,False,56.81,34.09,"[0.5776961, 0.5710784, 0.56397057, 0.5629902, ...","[0.11176600647627982, 0.0832705908633216, 0.20...","[-350.6411699488833, 76.15949173451945, 3.8318...","[151.86167025001237, 111.03726806371903, 146.1..."
2019-08-02 20:55:37,1300001,False,56.75,34.18,"[0.5897059, 0.5897059, 0.59387255, 0.6004902, ...","[0.09971700226105876, 0.07125987352405888, 0.1...","[-364.4164596121262, 94.39407039067773, 0.7766...","[76.72127394608977, 54.66498033200147, 31.0858..."
2019-08-02 21:11:37,1300001,False,56.75,34.21,"[0.59068626, 0.595098, 0.60269606, 0.6022059, ...","[0.14414751545603455, 0.12432094886995068, 0.1...","[-373.6403367366103, 84.52025568332569, 0.5508...","[158.66819376120802, 99.0386932970312, 84.2150..."
...,...,...,...,...,...,...,...,...
2019-08-03 21:25:37,1300001,False,55.68,34.15,"[0.5754902, 0.5715686, 0.5588235, 0.55710787, ...","[0.056908112580822054, 0.0689487417652858, 0.1...","[-363.26793389563426, 83.70740543719504, 1.276...","[157.2730706654439, 119.01149711448284, 73.813..."
2019-08-03 21:43:37,1300001,False,55.75,34.09,"[0.60539216, 0.60514706, 0.5921569, 0.595098, ...","[0.17881282780045848, 0.13517365079102106, 0.1...","[-340.2865850217681, 63.439747523888634, -1.26...","[50.859583042176496, 25.17963425025422, 33.853..."
2019-08-03 21:59:37,1300001,False,55.93,34.18,"[0.62058824, 0.6276961, 0.61446077, 0.59044117...","[0.12176968970515319, 0.13142249378404114, 0.1...","[-328.6537645279214, 63.60982191354608, -13.68...","[558.6972224075247, 206.94427968116545, 61.783..."


## k-mean clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# X_fft_values = StandardScaler().fit_transform(np.stack(df_data['fft'].values))
# X_temperature = StandardScaler().fit_transform(np.array(df_data['temperature'].values).reshape(-1, 1))
# X_humidity = StandardScaler().fit_transform(np.array(df_data['humidity'].values).reshape(-1, 1))
X_mfcc_avg = StandardScaler().fit_transform(np.stack(df_data['mfcc_avg'].values))
X_mfcc_var = StandardScaler().fit_transform(np.stack(df_data['mfcc_var'].values))
X_lpcs = StandardScaler().fit_transform(np.stack(df_data['lpcs'].values))

In [ ]:
X_temperature = np.array(df_data['temperature'].values)
X_humidity = np.array(df_data['humidity'].values)
X_mfcc_avg = np.stack(df_data['mfcc_avg'].values)
X_mfcc_var = np.stack(df_data['mfcc_var'].values)
X_lpcs = np.stack(df_data['lpcs'].values)

In [ ]:
X = []
for idx, samples in enumerate(X_lpcs):
    X.append(np.append(samples, np.append(X_temperature[idx], X_humidity[idx])))

In [ ]:
X = X_lpcs

In [ ]:
X = StandardScaler().fit_transform(X)

In [ ]:
X_mfcc_con = np.concatenate((X_mfcc_avg, X_mfcc_var), axis = 1)
X_mfcc_con = StandardScaler().fit_transform(X_mfcc_con)

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit( )
predicted = kmeans.predict(X_fft_values)

In [ ]:
X_fft_values.shape

In [ ]:
predicted_kmean = sum(predicted)
actual = int(sum(df_data['night_label'].values))
size_data = df_data.shape[0]
print(f'Predicted with kmean: {predicted_kmean} vs {actual}/{size_data}')

In [ ]:
# PCA
from sklearn.decomposition import PCA

reduced = PCA(n_components=2).fit_transform(X)

In [ ]:
X_reduced = []
for idx, pca_reduced in enumerate(reduced): 
    X_reduced.append(np.append(pca_reduced, np.append(X_temperature[idx], X_humidity[idx])))

X_reduced = StandardScaler().fit_transform(X_reduced)

In [ ]:
reduced_sec = PCA(n_components=2).fit_transform(X_reduced)

In [ ]:
# TSNE
from sklearn.manifold import TSNE

reduced = TSNE(n_components=2, perplexity=100, learning_rate=500, verbose=1).fit_transform(X_mfcc_avg)
reduced_sec = reduced

In [ ]:
%matplotlib notebook

colors = ['red', 'green']

x = [data[0] for data in reduced]
y = [data[1] for data in reduced]
values = [colors[int(data)] for data in df_data['night_label'].values]
scatter = plt.scatter(x, y,c=values, alpha = 0.3)
#plt.legend([1, 0], ['night', 'day'])
plt.show()


In [ ]:
df_data.head()

In [ ]:
# Import dependencies
import plotly
import plotly.graph_objs as go

colors = ['red', 'green', 'blue', 'yellow']

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

colors = [data for data in df_data['night_label'].values]
# Configure the trace.
trace = go.Scatter3d(
    x=[data[0] for data in reduced],  # <-- Put your data instead
    y=[data[1] for data in reduced],  # <-- Put your data instead
    z=[data[2] for data in reduced],  # <-- Put your data instead
    mode='markers',
    marker={
        'size': 10,
        'opacity': 0.8,
        'color': colors
    }
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]

plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
plotly.offline.iplot(plot_figure)

# MFCC

This block assumes that we have data in <code>df_data</code> variable.

In [ ]:
import librosa
import librosa.display

data_mfcc_labeled = []
night_timestamps = df_data.between_time("23:00", "3:30").index.values.tolist()
night_timestamps = pd.to_datetime(night_timestamps)

In [ ]:
df_data_labeled = df_data

In [ ]:
df_data_labeled.head()

In [ ]:
df_data_labeled['labeled'] = [int(index in night_timestamps) for index in df_data.index.values]

In [ ]:
for index, row in tqdm(df_data.iterrows(), total=df_data.shape[0]):
    full_mfccs = librosa.feature.mfcc(y=row['samples'], sr=3000, n_fft=512, hop_length=256, n_mfcc=14)
    data_mfcc_labeled.append([np.mean(full_mfccs,axis=1), row['temperature'], row['humidity'],
                              int(index in night_timestamps)])

# AUTOENCODER - BASIC

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as utils
import numpy as np
import pandas as pd

from tqdm import tqdm

In [ ]:
DATA_INIT = True
import glob 

auto_hives = [1300001, 1300002, 1400001, 1400002]
if DATA_INIT:
    auto_max_sample_value = 0
    auto_hive_sounds = []
    auto_hive_timestamps = []
    for idx, hive_id in enumerate(auto_hives):
        print(f"Data preparation for hive: {hive_id}")

        auto_sound_files = [f for f in glob.glob(f"measurements\\{hive_id}\\sound*.csv")]
        auto_sound_hive_list = []
        for file in tqdm(auto_sound_files):
            df_samples = pd.read_csv(file)
            pd_timestamp = pd.to_datetime(file.split("sound-")[1].split(".csv")[0], format='%Y-%m-%dT%H-%M-%S')
            if(len(df_samples.index) == 3000 and max(df_samples['samples'].values) < 4500):
                np_samples = np.array(df_samples['samples'].values, dtype="float32")
                np_samples = np_samples / 4080
                auto_hive_sounds.append([pd_timestamp, np_samples])
    np.random.shuffle(auto_hive_sounds)
    sound_pd = pd.DataFrame(auto_hive_sounds, columns=['timestamp', 'samples'])
    auto_df_data = sound_pd.set_index('timestamp')
    auto_df_data = sound_pd
else:
    sound_pd = np.load(f"{hives_ids[0]}-data.npy", allow_pickle=True)
    print(f"Loaded dataset with size {len(hive_sounds)}.")

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'We are running on : {device}')

num_epochs = 10
learning_rate = 1e-5
BATCH_SIZE = 30

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(3000, 2048),
            nn.ReLU(True),
            nn.Linear(2048, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 512),
            nn.ReLU(True),
            nn.Linear(512, 256)
        )
        self.decoder = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 2048),
            nn.ReLU(True),
            nn.Linear(2048, 3000), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
tensor_data = torch.FloatTensor([auto_df_data['samples'].values])
tensor_data.shape

In [ ]:
model = autoencoder()
model.to(device)

losses = []

def train(net):
    criterion = nn.MSELoss(reduction='sum')
    optimizer = torch.optim.RMSprop(net.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    for epoch in range(num_epochs):
        train_loss = 0
        for i in tqdm(range(0, tensor_data.size()[1], BATCH_SIZE)):
            batch_X = tensor_data[:, i:i+BATCH_SIZE, :]
            batch_X = batch_X.to(device)

            net.zero_grad()
            
            output = net(batch_X)
            loss = criterion(output, batch_X)
            loss.backward()
            optimizer.step()
    
            train_loss += loss.item()
        # ===================log========================
        loss_per_epoch = train_loss/tensor_data.size()[1]
        print(f"epoch [{epoch}/{num_epochs}], loss:{loss_per_epoch}")
        losses.append(loss_per_epoch)

In [ ]:
train(model)

In [ ]:
%matplotlib notebook  
import matplotlib.pyplot as plt


plt.plot(losses)
plt.title("Autoencoder loss")
plt.show()

In [ ]:
torch.save(model.state_dict(), 'autoencoder-06-11-2019-model.pth')

In [ ]:
sound_tensors = torch.FloatTensor([df_data_labeled['samples'].values])

In [ ]:
sss = list(zip(sound_tensors, df_data_labeled['labeled'].values))

In [ ]:
with torch.no_grad():
    sound_tensors = sound_tensors.to(device)
    output = model.encoder(sound_tensors)
    output = output.squeeze(0)

In [ ]:
output.shape

In [ ]:
output_cpu = output.cpu()

In [ ]:
labels_list_val = df_data_labeled['labeled'].values

In [ ]:
len(labels)

# Basic classification PCA

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
#standarized_mfcc_avg = StandardScaler().fit_transform([data[0] for data in data_mfcc_labeled])
#standarized_temp = StandardScaler().fit_transform([[data[1]] for data in data_mfcc_labeled])
#standarized_hum = StandardScaler().fit_transform([[data[2]] for data in data_mfcc_labeled])

standarized_autoencoder = StandardScaler().fit_transform(output.cpu())

#zipped_data = list(zip(standarized_mfcc_avg, standarized_temp, standarized_hum))
#merged_data = [np.concatenate(list_to_con) for list_to_con in zipped_data]
#standarized_merged_data = StandardScaler().fit_transform(merged_data)

pca = PCA(n_components=2)
pc_data = pca.fit_transform(standarized_autoencoder)

# Basic classification t-sne

In [ ]:
import numpy as np
from sklearn.manifold import TSNE

X_embedded = TSNE(n_components=2).fit_transform(output_cpu)

# Visualize

In [ ]:
len(pc_data[0])

In [ ]:
#pc_data_labeled = list(zip(pc_data, [mfcc_data[3] for mfcc_data in data_mfcc_labeled]))
colors = ['red', 'green', 'blue', 'yellow']
labels = ['day', 'night']

fig = plt.figure()

ax = fig.add_subplot(1, 1, 1)

# ax.scatter(x=[data[0][0] for data in pc_data_labeled],
#            y=[data[0][1] for data in pc_data_labeled],
#            c=[colors[int(data[1])] for data in pc_data_labeled],
#           alpha=0.3) 
ax.scatter(x=[data[0] for data in pc_data],
           y=[data[1] for data in pc_data],
           #c=[colors[data] for data in labels_list_val],
          alpha=0.3)
ax.legend()

plt.title("Autoencoder scatter plot")
plt.show()

### Evaluation

In [ ]:
model.load_state_dict(torch.load('autoencoder-06-11-2019-model.pth'))
model.to(device)

In [ ]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# load the training and test datasets
train_data = datasets.MNIST(root='mnist', train=True,
                                   download=False, transform=transform)
test_data = datasets.MNIST(root='mnist', train=False,
                                  download=False, transform=transform)

# Create training and test dataloaders

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

import matplotlib.pyplot as plt
%matplotlib inline
    
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()

# get one image from the batch
img = np.squeeze(images[0])

fig = plt.figure(figsize = (5,5)) 
ax = fig.add_subplot(111)
ax.imshow(img, cmap='gray')

import torch.nn as nn
import torch.nn.functional as F

# define the NN architecture
class Autoencoder(nn.Module):
    def __init__(self, encoding_dim):
        super(Autoencoder, self).__init__()
        ## encoder ##
        # linear layer (784 -> encoding_dim)
        self.fc1 = nn.Linear(28 * 28, encoding_dim)
        
        ## decoder ##
        # linear layer (encoding_dim -> input size)
        self.fc2 = nn.Linear(encoding_dim, 28*28)
        

    def forward(self, x):
        # add layer, with relu activation function
        x = F.relu(self.fc1(x))
        # output layer (sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.fc2(x))
        return x

# initialize the NN
encoding_dim = 32
model = Autoencoder(encoding_dim).to(device)
print(model)

# specify loss function
criterion = nn.MSELoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# number of epochs to train the model
n_epochs = 20


for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data in train_loader:
        # _ stands in for labels, here
        images, _ = data
        # flatten images
        images = images.view(images.size(0), -1).to(device)
        print(images.shape)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(images)
        # calculate the loss
        loss = criterion(outputs, images)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        print(images.size(0))
        train_loss += loss.item()*images.size(0)
            
    # print avg training statistics 
    train_loss = train_loss/len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, 
        train_loss
        ))

In [ ]:
def lpc(y, m):
    "Return m linear predictive coefficients for sequence y using Levinson-Durbin prediction algorithm"
    #step 1: compute autoregression coefficients R_0, ..., R_m
    R = [y.dot(y)] 
    if R[0] == 0:
        return [1] + [0] * (m-2) + [-1]
    else:
        for i in range(1, m + 1):
            r = y[i:].dot(y[:-i])
            R.append(r)
        R = np.array(R)
    #step 2: 
        A = np.array([1, -R[1] / R[0]])
        E = R[0] + R[1] * A[1]
        for k in range(1, m):
            if (E == 0):
                E = 10e-17
            alpha = - A[:k+1].dot(R[k+1:0:-1]) / E
            A = np.hstack([A,0])
            A = A + alpha * A[::-1]
            E *= (1 - alpha**2)
        return A